In [1]:
#libraries for model
import pickle
import numpy as np
import os
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


In [2]:
# Function to load the cifar dataset manually
def loadcifar10(file_path):
    with open(file_path, 'rb') as file:
        batch = pickle.load(file, encoding='latin1')
        data = batch['data']
        labels = batch['labels']

        # Reshapeing
        data = data.reshape(-1, 3, 32, 32)
        data = data.transpose(0, 2, 3, 1)
        
        return data, np.array(labels)

In [3]:
# Function to load training data
def loadcifar10data(dataset_dir):
    train_data = []
    train_labels = []
    
    # Load all 5 training batches
    for i in range(1, 6):
        file_path = os.path.join(dataset_dir, f"data_batch_{i}")
        data, labels = loadcifar10(file_path)
        train_data.append(data)
        train_labels.append(labels)
    
    
    train_data = np.concatenate(train_data)
    train_labels = np.concatenate(train_labels)
    
    # Load test data
    test_data, test_labels = loadcifar10(os.path.join(dataset_dir, "test_batch"))
    
    return train_data, train_labels, test_data, test_labels


In [4]:
dataset_directory = '/Users/harikrishnans/Downloads/cifar-10-batches-py'

In [5]:
train_data, train_labels, test_data, test_labels = loadcifar10data(dataset_directory)

In [6]:
print(f'Training data shape: {train_data.shape}')
print(f'Training labels shape: {train_labels.shape}')
print(f'Test data shape: {test_data.shape}')
print(f'Test labels shape: {test_labels.shape}')

Training data shape: (50000, 32, 32, 3)
Training labels shape: (50000,)
Test data shape: (10000, 32, 32, 3)
Test labels shape: (10000,)


In [7]:
#normalising
train_data = train_data.astype('float32') / 255.0
test_data = test_data.astype('float32') / 255.0

In [8]:
#categorical encoding
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

In [9]:
model = Sequential()

In [10]:
# Add Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [11]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
# Train the model
model.fit(train_data, train_labels, epochs=10, batch_size=64, validation_data=(test_data, test_labels))

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - accuracy: 0.3050 - loss: 1.8789 - val_accuracy: 0.5348 - val_loss: 1.3087
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 12s 15ms/step - accuracy: 0.4978 - loss: 1.4029 - val_accuracy: 0.5860 - val_loss: 1.1698
Epoch 3/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.5532 - loss: 1.2565 - val_accuracy: 0.6090 - val_loss: 1.1040
Epoch 4/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.5803 - loss: 1.1815 - val_accuracy: 0.6311 - val_loss: 1.0584
Epoch 5/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.6110 - loss: 1.1124 - val_accuracy: 0.6422 - val_loss: 1.0146
Epoch 6/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.6245 - loss: 1.0617 - val_accuracy: 0.6638 - val_loss: 0.9647
Epoch 7/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.6424 - loss: 1.0212 - val_accuracy: 0.6738 - val_loss: 0.9264
Epoch 8/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - accuracy: 0.6590 - loss: 0.9641 - 

In [13]:
test_loss, test_acc = model.evaluate(test_data, test_labels)

print(f'Test accuracy: {test_acc:.4f}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6850 - loss: 0.9094
Test accuracy: 0.6809


In [14]:
model.save('/Users/harikrishnans/Downloads/recog1.keras')


In [15]:
#Example usage
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

model = load_model('/Users/harikrishnans/Downloads/recog.keras')

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


img = image.load_img('/Users/harikrishnans/Downloads/dog1.jpeg', target_size=(32, 32))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0) / 255.0

prediction = model.predict(img)
print(f'Predicted class: {classes[np.argmax(prediction)]}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Predicted class: dog


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/saving/saving_lib.py:418: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))
